## 카메라 클라이언트
### Queue 사용해야 한다.
### 쓰레드 기반이므로, 주의

In [1]:
import socket
import time
import struct
import pandas as pd
import pickle
import cv2
import numpy as np
import time
import argparse
from threading import Thread, enumerate, Event
from queue import Queue

In [2]:
# 쓰레드 멈추기 위해 이벤트 사용한다.
event = Event()

def thread_socket(frame_queue):
    HOST_RPI = '127.0.0.1'
    PORT=8089

    client_cam = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_cam.connect((HOST_RPI, PORT))

    while True:
        cmd = 12
        cmd_byte = struct.pack('!B', cmd)
        client_cam.sendall(cmd_byte)
        print('sent .. ' + str(cmd))
        data_len_bytes = client_cam.recv(4)
        print('len : ' + str(len(data_len_bytes)))
        data_len = struct.unpack('!L', data_len_bytes)
        print('data len : ' + str(data_len[0]))
        frame_data = client_cam.recv(data_len[0], socket.MSG_WAITALL)
        print('real type : ' + str(type(frame_data)) + ',' + str(len(frame_data)))
        frame_queue.put(frame_data)
        if event.is_set():
            client_cam.close()
            return
            
    client_cam.shutdown()
    client_cam.close()

In [3]:
# Queue 만든다.
frame_queue = Queue()

Thread(target=thread_socket, daemon=True, args=(frame_queue,)).start()

while True:
    result = frame_queue.get()
    frame = pickle.loads(result)
    #frame3 = cv2.imdecode(frame, cv2.IMREAD_COLOR)
    frame3 = cv2.imdecode(frame, -1)
    cv2.imshow('frameX', frame3)
    if cv2.waitKey(1) == 27:
        # 쓰레드를 죽인다.
        event.set()
        break;

cv2.destroyAllWindows()

print('.......... see ? ..........' + str(frame3.shape))

sent .. 12
len : 4
data len : 7603
real type : <class 'bytes'>,7603
sent .. 12
len : 4
data len : 7554
real type : <class 'bytes'>,7554
sent .. 12
len : 4
data len : 7554
real type : <class 'bytes'>,7554
sent .. 12
len : 4
data len : 7452
real type : <class 'bytes'>,7452
sent .. 12
len : 4
data len : 7426
real type : <class 'bytes'>,7426
sent .. 12
len : 4
data len : 7426
real type : <class 'bytes'>,7426
sent .. 12
len : 4
data len : 7426
real type : <class 'bytes'>,7426
sent .. 12
len : 4
data len : 7376
real type : <class 'bytes'>,7376
sent .. 12
len : 4
data len : 7376
real type : <class 'bytes'>,7376
sent .. 12
len : 4
data len : 7376
real type : <class 'bytes'>,7376
sent .. 12
len : 4
data len : 7340
real type : <class 'bytes'>,7340
sent .. 12
len : 4
data len : 7340
real type : <class 'bytes'>,7340
sent .. 12
len : 4
data len : 7340
real type : <class 'bytes'>,7340
sent .. 12
len : 4
data len : 7321
real type : <class 'bytes'>,7321
sent .. 12
len : 4
data len : 7321
real type : <